# Задание 3

Реализуйте регулеризованый LDA с двумя параметрами из лекции. Сравните его с логистической рергерссией (осуществите тюнинг гиперпараметров для обоих моделей) на публичных датасетах (хотя бы 5) из UCI (https://archive.ics.uci.edu/).

**Дополнительно**: постарайтесь найти датасеты, где LDA работает лучше и попробуйте объяснить почему.

Подсказка: если данных достаточно много, то LDA и LogReg не будут отличаться даже если предположения LDA выполняются - попробуйте уменьшать обучающую выборку чтобы найти ситуации, когда LDA работает лучше. Если не получается с публичными датасетами - сгенерируйте (в этом случае посчитайте bias и variance для LDA и LorReg).

Скоро тут появится решение. Пока его тут нет =(